# Student Loan

# Setup

Run this cell **once** if you are in Google Colab:

In [ ]:
import sys

if "google.colab" in sys.modules:
    !rm -rf sample_data
    !git clone https://github.com/HMellor/personal-finance-uk.git
    !mv personal-finance-uk/* .
    !rm -r personal-finance-uk

    from google.colab import output

    output.enable_custom_widget_manager()

Then you can install the Python requirements:

In [ ]:
%pip install -q -r requirements.txt

## Interactive Figure

In [ ]:
import pandas as pd
from utils.loan import simulate_repayment
from utils.ui import InteractiveFigure
from utils.tax import net_present_value
from plotly.graph_objects import FigureWidget
import plotly.express as px


def plot(
    fig: FigureWidget,
    data: pd.DataFrame,
    inflation_rate: float,
    instant_repayment: float,
):
    """
    Plots the loan repayment data on the provided FigureWidget.

    Parameters:
    fig (FigureWidget): The Plotly figure widget to update.
    data (pd.DataFrame): The DataFrame containing loan repayment data.
    inflation_rate (float): The annual inflation rate (provided by InteractiveFigure.inputs).
    instant_repayment (float): The instant repayment amount (provided by provided by InteractiveFigure.inputs).
    """
    # Calculate discount reate and net persent value (NPV)
    discount_rate = inflation_rate / 12
    loan_npv = net_present_value(data["loan active"], discount_rate=discount_rate)[-1]

    # Calculate active monthly and annual repayments
    monthly_repayments_active = data[
        ["salary repayment active", "extra repayment active"]
    ].sum("columns")
    annual_repayments_active = monthly_repayments_active.groupby(data.index.year).sum()
    total_repayment_active_npv = (
        net_present_value(annual_repayments_active, discount_rate=inflation_rate).sum()
        + instant_repayment
    )

    # Calculate passive monthly and annual repayments
    monthly_repayments_passive = data[
        ["salary repayment passive", "extra repayment passive"]
    ].sum("columns")
    annual_repayments_passive = monthly_repayments_passive.groupby(
        data.index.year
    ).sum()
    total_repayment_passive_npv = net_present_value(
        annual_repayments_passive, discount_rate=inflation_rate
    ).sum()

    # Calculate interest saved
    interest_saved = total_repayment_passive_npv - total_repayment_active_npv

    # Create the title text
    title_text = (
        f"Final balance NPV: £{loan_npv:,.2f}, "
        f"Repayment months: {data['loan active'].argmin()}, "
        f"Total paid NPV: £{total_repayment_active_npv:,.2f}, "
        f"Interest saved: £{interest_saved:,.2f}"
    )

    # Prepare data for plotting
    data = data.drop(["gross passive", "extra repayment passive"], axis="columns")
    colors = px.colors.qualitative.Plotly
    color_discrete_sequence = colors[1:5] + colors[:6]
    lines = px.line(
        data,
        x=data.index,
        y=data.columns,
        color_discrete_sequence=color_discrete_sequence,
    )

    # Set line styles
    dash = ["dash"] * 4 + ["solid"] * 6
    for line, dash in zip(lines.data, dash):
        line.line.dash = dash

    # Update the figure with new data
    with fig.batch_update():
        if not fig.data:
            fig.update_yaxes(tickprefix="£", nticks=10, rangemode="nonnegative")
            fig.add_traces(
                lines.data,
                rows=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                cols=[1, 1, 2, 2, 1, 1, 1, 2, 2, 2],
            )
        else:
            for old_data, new_data in zip(fig.data, lines.data):
                old_data.x = new_data.x
                old_data.y = new_data.y
        # Update title
        fig.update_layout(title_text=title_text, title_x=0.5)


# Initialise and display the interactive figure
InteractiveFigure(plot=plot, update=simulate_repayment).app